In [14]:
!pip install sklearn

Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
    Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'


    ERROR: Command errored out with exit status 1:
     command: 'C:\Final_project_ML_IBM\security-log-threat-detection\gcn_env\Scripts\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\shash\\AppData\\Local\\Temp\\pip-install-cipa2xm2\\sklearn_8eeaf7cb9fb64f15b4925b26881ff136\\setup.py'"'"'; __file__='"'"'C:\\Users\\shash\\AppData\\Local\\Temp\\pip-install-cipa2xm2\\sklearn_8eeaf7cb9fb64f15b4925b26881ff136\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\shash\AppData\Local\Temp\pip-pip-egg-info-vhhiq9x3'
         cwd: C:\Users\shash\AppData\Local\Temp\pip-install-cipa2xm2\sklearn_8eeaf7cb9fb64f15b4925b26881ff136\
    Complete output (15 lines):
    The 'sklearn' PyPI package is deprecated, use 's

In [15]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import StandardScaler

In [16]:
df = pd.read_csv("../data/processed/ssh_clean.csv")
df.head()

,timestamp,source,ip,component,pid,event_type,event_template,raw_message,is_login_attempt,is_success
0,2025-12-10 06:55:46,ssh,173.234.31.186,LabSZ,24200,E27,reverse mapping checking getaddrinfo for <*> [...,reverse mapping checking getaddrinfo for ns.ma...,0,0
1,2025-12-10 06:55:46,ssh,173.234.31.186,LabSZ,24200,E13,Invalid user <*> from <*>,Invalid user webmaster from 173.234.31.186,1,0
2,2025-12-10 06:55:46,ssh,NaN,LabSZ,24200,E12,input_userauth_request: invalid user <*> [prea...,input_userauth_request: invalid user webmaster...,1,0
3,2025-12-10 06:55:46,ssh,NaN,LabSZ,24200,E21,pam_unix(sshd:auth): check pass; user unknown,pam_unix(sshd:auth): check pass; user unknown,0,0
4,2025-12-10 06:55:46,ssh,173.234.31.186,LabSZ,24200,E19,pam_unix(sshd:auth): authentication failure; l...,pam_unix(sshd:auth): authentication failure; l...,0,0


In [17]:
ips = df['ip'].dropna().unique().tolist()
ip_to_idx = {ip: i for i, ip in enumerate(ips)}

num_nodes = len(ips)
print("Total nodes:", num_nodes)

Total nodes: 30


In [18]:
edge_list = []

for _, row in df.iterrows():
    if pd.isna(row['ip']):
        continue
    src = ip_to_idx[row['ip']]
    dst = ip_to_idx[row['ip']]
    edge_list.append([src, dst])

edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
edge_index.shape

torch.Size([2, 1734])

In [19]:
node_features = [[] for _ in range(num_nodes)]

for _, row in df.iterrows():
    if pd.isna(row['ip']):
        continue
    idx = ip_to_idx[row['ip']]
    node_features[idx].append([
        row['is_login_attempt'],
        row['is_success']
    ])

In [20]:
node_features = [[] for _ in range(num_nodes)]

for _, row in df.iterrows():
    if pd.isna(row['ip']):
        continue
    idx = ip_to_idx[row['ip']]
    node_features[idx].append([
        row['is_login_attempt'],
        row['is_success']
    ])

In [21]:
final_features = []

for feats in node_features:
    if len(feats) == 0:
        final_features.append([0, 0, 0])
    else:
        arr = np.array(feats)
        final_features.append([
            arr[:,0].sum(),
            arr[:,1].sum(),
            len(arr)
        ])

node_features = final_features

In [22]:
# ---- DEGREE FEATURE ----
degrees = np.zeros(num_nodes)

for src, dst in edge_list:
    degrees[src] += 1
    degrees[dst] += 1

for i in range(len(node_features)):
    node_features[i].append(degrees[i])

# ---- NORMALIZE FEATURES ----
scaler = StandardScaler()
x_np = scaler.fit_transform(node_features)
x = torch.tensor(x_np, dtype=torch.float)

print("Feature shape:", x.shape)

Feature shape: torch.Size([30, 4])


In [23]:
labels = [0] * num_nodes

for ip in df['ip'].dropna().unique():
    subset = df[df['ip'] == ip]
    failed = subset['is_login_attempt'].sum()

    if failed > 5:
        labels[ip_to_idx[ip]] = 1  # attacker
    else:
        labels[ip_to_idx[ip]] = 0  # benign

y = torch.tensor(labels, dtype=torch.long)
print("Attack count:", sum(labels))

Attack count: 9


In [24]:
class GCN(nn.Module):
    def __init__(self, in_channels, hidden=64, out_channels=2):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden)
        self.conv2 = GCNConv(hidden, out_channels)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        return x

In [25]:
model = GCN(in_channels=x.shape[1], hidden=64, out_channels=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)

In [26]:
for epoch in range(200):
    optimizer.zero_grad()
    out = model(x, edge_index)
    loss = F.cross_entropy(out, y)
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch}  Loss: {loss.item():.4f}")

Epoch 0  Loss: 0.7168
Epoch 20  Loss: 0.3041
Epoch 40  Loss: 0.2054
Epoch 60  Loss: 0.1532
Epoch 80  Loss: 0.1611
Epoch 100  Loss: 0.1425
Epoch 120  Loss: 0.1190
Epoch 140  Loss: 0.1036
Epoch 160  Loss: 0.0702
Epoch 180  Loss: 0.0852


In [27]:
pred = out.argmax(dim=1)
acc = (pred == y).sum().item() / len(y)
print("Accuracy:", acc)

Accuracy: 0.9333333333333333


In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)

In [28]:
y = torch.tensor(labels, dtype=torch.long)

In [29]:
import torch

num_nodes = x.shape[0]

# 80% train, 20% test
train_ratio = 0.8
num_train = int(train_ratio * num_nodes)

# shuffle indices
perm = torch.randperm(num_nodes)

train_idx = perm[:num_train]
test_idx = perm[num_train:]

print("Train nodes:", len(train_idx))
print("Test nodes:", len(test_idx))

# create boolean masks
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[train_idx] = True
test_mask[test_idx] = True


Train nodes: 24
Test nodes: 6


In [30]:
class GCN(nn.Module):
    def __init__(self, in_channels, hidden=64, out_channels=2):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden)
        self.conv2 = GCNConv(hidden, out_channels)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        return x


In [31]:
model = GCN(in_channels=x.shape[1], hidden=64, out_channels=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)

In [32]:
for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    
    out = model(x, edge_index)
    
    # compute loss ONLY on train nodes
    loss = F.cross_entropy(out[train_mask], y[train_mask])
    
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch}  Loss: {loss.item():.4f}")

Epoch 0  Loss: 0.5976
Epoch 20  Loss: 0.2593
Epoch 40  Loss: 0.1747
Epoch 60  Loss: 0.1835
Epoch 80  Loss: 0.1564
Epoch 100  Loss: 0.1668
Epoch 120  Loss: 0.0953
Epoch 140  Loss: 0.0699
Epoch 160  Loss: 0.0676
Epoch 180  Loss: 0.1096


In [33]:
model.eval()
out = model(x, edge_index)
pred = out.argmax(dim=1)

test_correct = (pred[test_mask] == y[test_mask]).sum().item()
test_total = test_mask.sum().item()

test_acc = test_correct / test_total
print("Test Accuracy:", test_acc)

Test Accuracy: 1.0
